In [23]:
import random
import math
import statistics

def parse_evrp_instance(filepath):
    """
    Parses an EVRP instance file to extract dimensions, coordinates,
    demands, depot ID, and client node IDs.
    """
    with open(filepath, 'r') as f:
        lines = f.readlines()

    dimension = 0
    stations_count = 0
    
    # Flags para controlar a leitura das seções
    node_coords_section = False
    demand_section = False
    stations_coord_section = False
    reading_depot_id = False # Nova flag para lidar com a linha do depósito
    
    coordenadas = {}
    demandas = {}
    depot_id = -1
    
    for line_idx, raw_line in enumerate(lines): # Usar enumerate para ter o índice da linha original
        line = raw_line.strip() # Stripped version for comparisons and processing
        if not line: # Ignora linhas em branco
            continue

        if line.startswith("DIMENSION:"):
            dimension = int(line.split(":")[1].strip())
        elif line.startswith("STATIONS:"):
            stations_count = int(line.split(":")[1].strip())
        elif line.startswith("NODE_COORD_SECTION"):
            node_coords_section = True
            demand_section = False
            stations_coord_section = False
            reading_depot_id = False # Reseta a flag
            continue
        elif line.startswith("DEMAND_SECTION"):
            demand_section = True
            node_coords_section = False
            stations_coord_section = False
            reading_depot_id = False # Reseta a flag
            continue
        elif line.startswith("STATIONS_COORD_SECTION"):
            stations_coord_section = True
            node_coords_section = False
            demand_section = False
            reading_depot_id = False # Reseta a flag
            continue
        elif line.startswith("DEPOT_SECTION"):
            reading_depot_id = True # Ativa a flag para ler o ID na próxima iteração
            # Desativa flags de seção de dados para evitar parsear as linhas seguintes como dados de seção
            node_coords_section = False
            demand_section = False
            stations_coord_section = False
            continue # Continua para a próxima linha onde o ID estará
        elif line.startswith("EOF"):
            break

        # Lógica para ler o DEPOT ID APÓS a linha "DEPOT_SECTION"
        if reading_depot_id:
            try:
                depot_id = int(line) # A linha atual deve ser o ID do depósito
                reading_depot_id = False # Já lemos, desativa a flag
                # Se for -1, significa que o arquivo tem um formato de terminação diferente
                if depot_id == -1: 
                    break # Fim da seção do depósito
            except ValueError:
                # Caso a linha após DEPOT_SECTION não seja um número (erro no arquivo)
                print(f"Aviso: Esperado ID do depósito numérico, encontrado '{line}' após DEPOT_SECTION.")
                reading_depot_id = False # Para evitar loop infinito se a próxima não for número
            continue # Passa para a próxima linha para não processar o ID do depósito como outro dado

        # Processar as linhas dentro das seções ativas
        if node_coords_section and line[0].isdigit():
            parts = line.split()
            if len(parts) >= 3: 
                node_id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                coordenadas[node_id] = (x, y)
            # else: print(f"Aviso: Linha inesperada ou mal formatada na NODE_COORD_SECTION: '{line}'")

        elif demand_section and line[0].isdigit():
            parts = line.split()
            if len(parts) >= 2: 
                node_id = int(parts[0])
                demand = int(parts[1]) 
                demandas[node_id] = demand
            # else: print(f"Aviso: Linha inesperada ou mal formatada na DEMAND_SECTION: '{line}'")
        
        elif stations_coord_section and line[0].isdigit():
            # No arquivo E-n23-k3.evrp.txt, esta seção contém apenas IDs das estações, 
            # não suas coordenadas novamente (que já estão na NODE_COORD_SECTION).
            # Por isso, apenas ignoramos ou podemos armazenar os IDs das estações se necessário.
            pass

    # Identificar nós clientes: todos os nós que possuem demanda > 0
    # Assumimos que o nó do depósito tem demanda 0 (como no E-n23-k3.evrp.txt)
    client_nodes = [node_id for node_id, demand in demandas.items() if demand > 0]
    
    return dimension, stations_count, coordenadas, demandas, depot_id, client_nodes

In [24]:
# Calcular Matriz de Distâncias
def calcular_matriz_distancias(coordenadas):
    # Calcula a matriz de distâncias euclidianas entre todos os nós.
    distancias = {}
    for i in coordenadas:
        distancias[i] = {}
        for j in coordenadas:
            if i == j:
                distancias[i][j] = 0
            else:
                x1, y1 = coordenadas[i]
                x2, y2 = coordenadas[j]
                distancias[i][j] = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return distancias

In [25]:
def criar_individuo(num_clientes):
    # Cria um indivíduo (solução) binário aleatório.
    return [random.randint(0, 1) for _ in range(num_clientes)]

In [26]:
def criar_populacao_inicial(tamanho_populacao, num_clientes):
    # Cria a população inicial de indivíduos.
    return [criar_individuo(num_clientes) for _ in range(tamanho_populacao)]

In [27]:
def selecionar_pais(populacao, aptidoes):
    # Seleciona dois pais da população com base na aptidão (roleta).
    total_aptidao = sum(aptidoes)
    if total_aptidao == 0:
        return random.sample(populacao, 2)
    
    probabilidades = [apt / total_aptidao for apt in aptidoes]
    pai1 = random.choices(populacao, weights=probabilidades, k=1)[0]
    pai2 = random.choices(populacao, weights=probabilidades, k=1)[0]
    return pai1, pai2

In [28]:
def crossover(pai1, pai2):
    # Realiza o crossover de um ponto entre dois pais.
    ponto_corte = random.randint(1, len(pai1) - 1)
    filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
    filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
    return filho1, filho2

In [29]:
def mutacao(individuo, taxa_mutacao):
    # Realiza a mutação em um indivíduo.
    for i in range(len(individuo)):
        if random.random() < taxa_mutacao:
            individuo[i] = 1 - individuo[i]
    return individuo

In [30]:
# Função de Aptidão
def calcular_aptidao_evrp(individuo, distancias_map, depot_node_id, client_nodes_list):
    """
    Calcula a aptidão de um indivíduo para o EVRP simplificado.
    A aptidão é o inverso da distância total.
    """
    global avaliacoes_count
    avaliacoes_count += 1

    rotas = []
    current_route = [depot_node_id] # Todas as rotas começam no depósito
    
    # Certificar-se de que client_nodes_list está ordenada para consistência com o cromossomo
    sorted_client_nodes = sorted(client_nodes_list)

    for i in range(len(individuo)):
        cliente_id = sorted_client_nodes[i]
        
        current_route.append(cliente_id)

        # Se o bit correspondente é 1 (retorno ao depósito) ou se é o último cliente na lista
        if individuo[i] == 1 or (i == len(individuo) - 1):
            current_route.append(depot_node_id) # Retorna ao depósito
            rotas.append(current_route)
            current_route = [depot_node_id] # Inicia uma nova rota a partir do depósito

    # Calcular a distância total das rotas
    total_distance = 0
    for route in rotas:
        for k in range(len(route) - 1):
            total_distance += distancias_map[route[k]][route[k+1]]
    
    if total_distance == 0:
        return 0 # Evitar divisão por zero
    
    return 1 / total_distance # Maximizar aptidão == Minimizar distância

In [31]:
def algoritmo_genetico_evrp_binario_instancia(
    tamanho_populacao, 
    taxa_mutacao, 
    max_avaliacoes,
    num_clientes,
    distancias_map,
    depot_node_id,
    client_nodes_list
):
    global avaliacoes_count
    avaliacoes_count = 0 

    populacao = criar_populacao_inicial(tamanho_populacao, num_clientes)
    melhor_solucao = None
    melhor_aptidao = -1
    
    geracao = 0
    # A condição de parada é baseada no total de avaliações
    while avaliacoes_count < max_avaliacoes:
        aptidoes = []
        for ind in populacao:
            apt = calcular_aptidao_evrp(ind, distancias_map, depot_node_id, client_nodes_list)
            aptidoes.append(apt)
            if apt > melhor_aptidao:
                melhor_aptidao = apt
                melhor_solucao = list(ind) # Faz uma cópia para evitar referência

        nova_populacao = []
        # Elitismo: Garante que o melhor indivíduo não seja perdido
        # Adiciona o melhor indivíduo atual à próxima geração se ele existir
        if melhor_solucao:
            nova_populacao.append(list(melhor_solucao)) # Adiciona uma cópia

        # Preenche o restante da nova população
        # Garante que as avaliações não excedam o limite
        while len(nova_populacao) < tamanho_populacao and avaliacoes_count < max_avaliacoes:
            pai1, pai2 = selecionar_pais(populacao, aptidoes)
            filho1, filho2 = crossover(pai1, pai2)
            
            nova_populacao.append(mutacao(filho1, taxa_mutacao))
            if len(nova_populacao) < tamanho_populacao:
                nova_populacao.append(mutacao(filho2, taxa_mutacao))
        
        populacao = nova_populacao
        geracao += 1
        
        # Imprimir progresso de vez em quando
        # if geracao % 50 == 0:
        #    print(f"Geração {geracao}, Avaliações: {avaliacoes_count}, Melhor Aptidão: {melhor_aptidao:.4f}")

    # Ao final, re-calculamos a distância da melhor solução para retornar o valor minimizado
    distancia_minima = 1 / melhor_aptidao if melhor_aptidao > 0 else float('inf')
    
    # Reconstruir as rotas para a melhor solução final
    rotas_finais = []
    if melhor_solucao:
        sorted_client_nodes = sorted(client_nodes_list)
        current_route = [depot_node_id]
        for i in range(len(melhor_solucao)):
            cliente_id = sorted_client_nodes[i]
            current_route.append(cliente_id)
            if melhor_solucao[i] == 1 or (i == len(melhor_solucao) - 1):
                current_route.append(depot_node_id)
                rotas_finais.append(current_route)
                current_route = [depot_node_id]

    return melhor_solucao, distancia_minima, rotas_finais, avaliacoes_count

In [ ]:
def executar_instancias(FILE_PATH):
    # Analisa a instância
    dimension, stations_count, coordenadas_globais, demandas_globais, depot_id_global, client_nodes_global = parse_evrp_instance(
        FILE_PATH)

    # Recalcular N (total de nós) e NUM_CLIENTES
    # N é o número total de nós no grafo: dimension.
    # NUM_CLIENTES é o número de clientes que precisam ser visitados, ou seja, len(client_nodes_global)
    NUM_CLIENTES_INSTANCIA = len(client_nodes_global)
    N_INSTANCIA = dimension  # Total number of nodes (depot + customers + stations)

    distancias_instancia = calcular_matriz_distancias(coordenadas_globais)

    # --- Parâmetros do AG para a Instância ---
    TAMANHO_POPULACAO = 100
    TAXA_MUTACAO = 0.02

    # n = DIMENSION do problema
    n_param = dimension  # Usamos a DIMENSION para o 'n' da condição de parada

    MAX_AVALIACOES_INSTANCIA = 25000 * n_param

    print(f"Instância carregada: {FILE_PATH}")
    print(f"Dimensão (total de nós): {dimension}")
    print(f"Número de Clientes (com demanda > 0): {NUM_CLIENTES_INSTANCIA}")
    print(f"ID do Depósito: {depot_id_global}")
    print(f"Número máximo de avaliações (25000 * n): {MAX_AVALIACOES_INSTANCIA}")

    NUM_EXECUCOES = 20  # Número de vezes que o AG será executado

    print(
        f"Iniciando {NUM_EXECUCOES} execuções do AG para a instância {FILE_PATH}")
    print(f"Número máximo de avaliações por execução: {MAX_AVALIACOES_INSTANCIA}")

    resultados_distancias = []
    for i in range(NUM_EXECUCOES):
        print(f"\n--- Execução {i+1}/{NUM_EXECUCOES} ---")
        # Reseta o contador de avaliações global para cada execução
        global avaliacoes_count
        avaliacoes_count = 0

        # Executa o AG
        best_solution, menor_distancia, rotas, total_avaliacoes = algoritmo_genetico_evrp_binario_instancia(
            TAMANHO_POPULACAO,
            TAXA_MUTACAO,
            MAX_AVALIACOES_INSTANCIA,
            NUM_CLIENTES_INSTANCIA,
            distancias_instancia,
            depot_id_global,
            client_nodes_global
        )
        resultados_distancias.append(menor_distancia)
        print(f"Melhor solução: {best_solution}")
        print(f"  Distância mínima encontrada nesta execução: {menor_distancia:.2f}")
        print(f"Rotas finais: {rotas}")
        print(f"  Total de avaliações nesta execução: {total_avaliacoes}")

    # --- Extração e Apresentação das Estatísticas ---
    print("\n--- Estatísticas das Execuções ---")
    print(f"Resultados das Distâncias ({NUM_EXECUCOES} execuções):")
    for i, dist in enumerate(resultados_distancias):
        print(f"  Execução {i+1}: {dist:.2f}")

    media = statistics.mean(resultados_distancias)
    desvio_padrao = statistics.stdev(resultados_distancias)
    maximo = max(resultados_distancias)
    minimo = min(resultados_distancias)

    print(f"\nMédia das Distâncias: {media:.2f}")
    print(
        f"Desvio Padrão das Distâncias: {desvio_padrao:.2f}")
    print(f"Valor Máximo da Distância: {maximo:.2f}")
    print(f"Valor Mínimo da Distância: {minimo:.2f}")
        
    # --- Bloco de Código para Escrever em Arquivo de Texto ---
    output_filename = FILE_PATH + ".txt"

    with open(output_filename, 'w') as f:
        f.write(f"--- Estatísticas das Execuções do AG Binário ---\n")
        f.write(f"Instância: {FILE_PATH}\n")
        f.write(f"Número de Execuções: {NUM_EXECUCOES}\n")
        f.write(f"Número máximo de avaliações por execução: {MAX_AVALIACOES_INSTANCIA}\n\n")
        
        f.write(f"Resultados das Distâncias Individuais:\n")
        for i, dist in enumerate(resultados_distancias):
            f.write(f"  Execução {i+1}: {dist:.2f}\n")

        f.write(f"\n--- Resumo Estatístico ---\n")
        f.write(f"Média das Distâncias: {media:.2f}\n")
        f.write(f"Desvio Padrão das Distâncias: {desvio_padrao:.2f}\n")
        f.write(f"Valor Máximo da Distância: {maximo:.2f}\n")
        f.write(f"Valor Mínimo da Distância: {minimo:.2f}\n")

        if FILE_PATH == 'E-n23-k3.evrp':
            optimal_value_t1 = 571.94
            optimal_value_t2 = 	571.94
            optimal_value_t3 = 	571.94

            f.write(f"\nValor Ótimo Time 1: {optimal_value_t1:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t1) / optimal_value_t1) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")

            f.write(f"\nValor Ótimo Time 2: {optimal_value_t2:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t2) / optimal_value_t2) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")

            f.write(f"\nValor Ótimo Time 1: {optimal_value_t3:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t3) / optimal_value_t3) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")
        elif FILE_PATH == 'E-n51-k5.evrp':
            optimal_value_t1 = 543.26
            optimal_value_t2 = 533.66
            optimal_value_t3 = 542.08

            f.write(f"\nValor Ótimo Time 1: {optimal_value_t1:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t1) / optimal_value_t1) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")

            f.write(f"\nValor Ótimo Time 2: {optimal_value_t2:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t2) / optimal_value_t2) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")

            f.write(f"\nValor Ótimo Time 1: {optimal_value_t3:.2f}\n")
            mean_diff_percent = ((media - optimal_value_t3) / optimal_value_t3) * 100
            f.write(f"Diferença percentual média: {mean_diff_percent:.2f}%\n")

    print(f"\nDados estatísticos salvos em '{output_filename}'")

In [33]:
INSTANCIAS = ['E-n23-k3.evrp', 'E-n51-k5.evrp']

for instancia in INSTANCIAS:
    executar_instancias(instancia)
    print("\n")

Instância carregada: E-n23-k3.evrp
Dimensão (total de nós): 23
Número de Clientes (com demanda > 0): 22
ID do Depósito: 1
Número máximo de avaliações (25000 * n): 575000
Iniciando 20 execuções do AG para a instância E-n23-k3.evrp
Número máximo de avaliações por execução: 575000

--- Execução 1/20 ---
Melhor solução: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
  Distância mínima encontrada nesta execução: 918.56
Rotas finais: [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 1]]
  Total de avaliações nesta execução: 575000

--- Execução 2/20 ---
Melhor solução: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Distância mínima encontrada nesta execução: 918.56
Rotas finais: [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 1]]
  Total de avaliações nesta execução: 575000

--- Execução 3/20 ---
Melhor solução: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 